<a href="https://colab.research.google.com/github/shrikantnarayankar15/hourse_rent_insaid/blob/master/hourse_rent_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EDA on House Rent


##1.Problem Statement
  here we are going to observe the house prices by its featues like ground_size, bedroom_number, garage have or not, neightbourhood, quality of residence, etc. and we will use different models to predict the output of houses i.e. house price.

##2.Importing Packages

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import boxcox_normmax, zscore
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import GradientBoostingRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.preprocessing import RobustScaler
from scipy.stats import boxcox_normmax, zscore
from multiprocessing import cpu_count
from lightgbm import LGBMRegressor
from scipy.special import boxcox1p
import matplotlib.pyplot as plt
from sklearn.svm import SVR
import pandas as pd
import numpy as np
#library
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import random as rnd

from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

# Importing Models
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Importing other tools
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, classification_report, make_scorer
from sklearn.metrics import accuracy_score, recall_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

##3.Data Preprocessing

In [0]:
#Loading Data
df = pd.read_csv('https://raw.githubusercontent.com/shrikantnarayankar15/hourse_rent_insaid/master/train.csv',index_col=0)
test_data = pd.read_csv('https://raw.githubusercontent.com/shrikantnarayankar15/hourse_rent_insaid/master/test.csv', index_col=0)

###1.Removing Outliers

In [0]:
sns.scatterplot(x="GrLivArea", y="SalePrice", data=df)

In [0]:
df = df[~(df.GrLivArea>4500)]

In [0]:
sns.scatterplot(x="GrLivArea", y="SalePrice", data=df)

###2.Combining test and training dataset
**test** dataset does not have independent variable i.e.output variable thats why while combining we are concatenating train dataset with test dataset without SalePrice feature.

In [0]:
X = pd.concat([df.drop("SalePrice", axis=1), test_data])

In [0]:
X.head()

df.head()

###3.data Description


*   Total 1460 rows are there
*   81 columns are there





In [0]:
X.describe()

In [0]:
# from  pandas_profiling import ProfileReport
# profile = ProfileReport(test_data, title='Pandas Profiling Report', html={'style':{'full_width':True}})
# # profile.to_notebook_iframe()
# profile.to_file(output_file="output.html")

###4.Checking Null Values

####1.Columns with high number of NaN

In [0]:
# missing values
w = X.isnull().sum()
w = w[w>0]
w.sort_values(ascending=False)[:5]

In [0]:
X.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'], axis=1, inplace=True)

we will drop this columns as they high nnumber of Null values

In [0]:
w = X.isnull().sum()
w = w[w>0]
w.sort_values(ascending=False)


**2**.filling null values
we Are filling null values with Neighborhood, we selected this feature to fill null values because area wise they have high chances of probablity that they may have same feature.

In [0]:
# LotFrontage and MasVnrArea are continous variables
conts_cols = ['LotFrontage', 'MasVnrArea', 'BsmtFullBath','BsmtHalfBath','TotalBsmtSF','BsmtUnfSF','BsmtFinSF2','BsmtFinSF1','GarageArea','GarageCars',]
X[conts_cols] = X.groupby("Neighborhood")[conts_cols].transform(lambda x: x.fillna(x.median()))

In [0]:
# fill the categorical variables
conts_cols = ['GarageCond','MSZoning', 'Functional','SaleType', 'KitchenQual',
       'Exterior2nd', 'Exterior1st','Utilities','GarageYrBlt','GarageFinish','GarageQual','GarageType','BsmtFinType2','BsmtExposure','BsmtFinType1','BsmtCond','BsmtQual','MasVnrType','Electrical']
X[conts_cols] = X.groupby("Neighborhood")[conts_cols].transform(lambda x: x.fillna(x.mode()[0]))

In [0]:
#check null values
w = X.isnull().sum()
w = w[w>0]
w.sort_values(ascending=False)

###5.Checking the 0 values

In [0]:
w=(X==0).sum()
w = w[w>0]
w.head()

As we can see the MasVnrArea have too many 0 values which is why we are going to remove it. also BsmtFinSF2 

In [0]:
(X==0).sum().nlargest(7)

In [0]:
X.drop(['BsmtFinSF2', 'MiscVal', 'PoolArea','LowQualFinSF'], axis=1,inplace=True)

###6.Combining features

In [0]:
X["TotalSF"] = X["GrLivArea"] + X["TotalBsmtSF"]
X["TotalPorchSF"] = X["OpenPorchSF"] + X["EnclosedPorch"] + X["3SsnPorch"] + X["ScreenPorch"]
X["TotalBath"] = X["FullBath"] + X["BsmtFullBath"] + 0.5 * (X["BsmtHalfBath"] + X["HalfBath"])

In [0]:
X.drop(['GrLivArea', 'TotalBsmtSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','FullBath','BsmtFullBath','BsmtHalfBath','HalfBath'],axis=1,inplace=True)

##4.EDA on hour-price

###1.Saleprice bar chart


In [0]:
#Type of house and price
df.groupby(['MSSubClass'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

from here we can say the type<br>
60-   2-STORY 1946 & NEWERare having highest saleprice<br>
after that 120, 75, 20 and so on 

In [0]:
# #is any year correlated to price?
# import seaborn as sns
# year_columns = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']
# df_year = df.loc[:,year_columns+['SalePrice']]

# fig, axs = plt.subplots(2,2,figsize=(10,10))
# axs = axs.ravel(order='F')
# dicto = {'null_values':[], 'null_value %':[], 'column_name':[]}
# for i,column in enumerate(year_columns):
#   sns.lineplot(x=column, y="SalePrice", data=df_year, ax=axs[i])
# # plt.tight_layout(pad=4.0)
# # axs = axs.ravel(order='F')


In [0]:
#Type of house and price
df.groupby(['MSZoning'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

Floating Village Residential are having higher sale price <br>
commercial are having cheaper rates

In [0]:
fig, axs = plt.subplots(1,2,figsize=(20,10))
axs = axs.ravel(order='F')
sns.scatterplot(x='LotFrontage', y="SalePrice", data=df,ax=axs[0])
sns.scatterplot(x='LotArea', y="SalePrice", data=df, ax=axs[1])


In [0]:
df.loc[:,['LotFrontage','LotArea','SalePrice']].corr()

From here we can say if lotarea and lotfrontage will increase sale price will increase too

In [0]:
df.groupby(['Street'])['SalePrice'].mean().plot.bar()

Street Paved have more sale price 

In [0]:
df.groupby(['LotShape'])['SalePrice'].mean().plot.bar()

In [0]:
df.groupby(['LandContour'])['SalePrice'].mean().plot.bar()

In [0]:
df.groupby(['Utilities'])['SalePrice'].mean().plot.bar()

In [0]:
df.groupby(['LotConfig'])['SalePrice'].mean().plot.bar()

In [0]:
df.groupby(['LandSlope'])['SalePrice'].mean().plot.bar()

In [0]:
df.groupby(['Neighborhood'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
df.groupby(['BldgType'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
df.groupby(['OverallQual'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()


In [0]:
df.groupby(['RoofStyle'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
df.groupby(['RoofMatl'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
df.groupby(['BsmtFullBath'])['SalePrice'].mean().sort_values(ascending=False).plot.bar()

In [0]:
# sns.scatterplot(x="GrLivArea", y="SalePrice", data=df)

In [0]:
from sklearn.preprocessing import StandardScaler
a = StandardScaler()

df_sclar = a.fit_transform(df[['SalePrice','LotArea']])

In [0]:
#removing this outliers
# X = X[~(X.GrLivArea>4500)]

##5.Model Evaluation

In [0]:
y_train = np.log(df["SalePrice"])


In [0]:
X = pd.get_dummies(X)


In [0]:
X_train = X.loc[df.index]
X_test = X.loc[test_data.index]

In [0]:
residuals = y_train - LinearRegression().fit(X_train, y_train).predict(X_train)
outliers = residuals[np.abs(zscore(residuals)) > 3].index
X_train = X_train.drop(outliers)
y_train = y_train.drop(outliers)
outliers

In [0]:
X_test.fillna(0,inplace=True)

In [0]:
from sklearn.linear_model import LinearRegression
# clf = LinearRegression()
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)


In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
# from sklearn.preprocessing import StandardScaler
# a = StandardScaler()
# X_train = a.fit_transform(X_train)
from sklearn.preprocessing import normalize


In [0]:
from sklearn.model_selection import train_test_split
X_th, x_tr, y_th,y_tr = train_test_split(X_train, y_train, test_size=0.2)

In [0]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import tree
from math import sqrt
clf = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls')
clf.fit(X_th, y_th)
y_pred = clf.predict(x_tr)
rms = sqrt(mean_squared_error(y_tr, y_pred))
print(rms)

In [0]:
model = linear_model.ElasticNetCV(alphas=np.logspace(-6, 6, 13))
model.fit(X_th, y_th)
y_pred = model.predict(x_tr)
rms = sqrt(mean_squared_error(y_tr, y_pred))
print(rms)

In [0]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_th, y_th)
y_pred = model.predict(x_tr)
rms = sqrt(mean_squared_error(y_tr, y_pred))
print(rms)

In [0]:
y_pred

In [0]:
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(degree=2)
# X_th = poly.fit_transform(X_th)

In [0]:
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor

#create a dictionary of our models
elastic = linear_model.ElasticNetCV(alphas=np.logspace(-6, 6, 13))
lasso = linear_model.LassoCV(alphas=np.logspace(-6, 6, 13))
ridge = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))
xgb = XGBRegressor()
estimators=[('Elastic',elastic ),('ridge',ridge), ('xgb',xgb)]
ensemble = VotingRegressor(estimators)
final_layer = StackingRegressor(
estimators=[('rf', RandomForestRegressor(random_state=42)),
('gbrt', GradientBoostingRegressor(random_state=42))],
final_estimator=linear_model.LassoCV()
)
reg = StackingRegressor(estimators=estimators,final_estimator=final_layer)
reg.fit(X_th, y_th)
#test our model on the test data
print(reg.score(x_tr, y_tr))
y_pred = reg.predict(x_tr)
rms = sqrt(mean_squared_error(y_tr, y_pred))
print(rms)

In [0]:
# from sklearn import linear_model
# clf = linear_model.ElasticNetCV(alphas=np.logspace(-6, 6, 13))
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [0]:
from sklearn.linear_model import ElasticNet
kf = KFold(n_splits=6, random_state=0, shuffle=True)
rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))
scorer = make_scorer(rmse, greater_is_better=False)
def random_search(model, grid, n_iter=100):
    n_jobs = max(cpu_count() -1, 1)
    search = RandomizedSearchCV(model, grid, n_iter, scorer, n_jobs=n_jobs, cv=kf, random_state=0, verbose=True)
    return search.fit(X_train, y_train)
ridge_search = random_search(Ridge(), {"alpha": np.logspace(-1, 2, 500)})
lasso_search = random_search(Lasso(), {"alpha": np.logspace(-5, -1, 500)})
elastic = random_search(ElasticNet(), {"alpha": np.logspace(-5, -1, 500)})
svr_search = random_search(SVR(), {"C": np.arange(1, 100), "gamma": np.linspace(0.00001, 0.001, 50), "epsilon": np.linspace(0.01, 0.1, 50)})
lgbm_search = random_search(LGBMRegressor(n_estimators=2000, max_depth=3), {"colsample_bytree": np.linspace(0.2, 0.7, 6), "learning_rate": np.logspace(-3, -1, 100)})
gbm_search = random_search(GradientBoostingRegressor(n_estimators=2000, max_depth=3), {"max_features": np.linspace(0.2, 0.7, 6), "learning_rate": np.logspace(-3, -1, 100)})
models = [search.best_estimator_ for search in [ridge_search, lasso_search, elastic,svr_search, lgbm_search, gbm_search]]
stack_search = random_search(StackingCVRegressor(models, Ridge(), cv=kf), {"meta_regressor__alpha": np.logspace(-3, -2, 500)}, n_iter=20)
models.append(stack_search.best_estimator_)
preds = [model.predict(X_test) for model in models]

In [0]:
preds = [model.predict(X_test) for model in models]
preds = np.average(preds, axis=0)


In [0]:
submission = pd.DataFrame({"Id": test_data.index.values, "SalePrice":np.exp(y_pred)})
submission.to_csv("submission_house_rent.csv", index=False)

train_data = pd.get_dummies(X)
train_data.head()

In [0]:
test_data = pd.get_dummies(test_data)

In [0]:
test_data.shape, train_data.shape

In [0]:
X = train_data.loc[:,train_data.columns != 'SalePrice']
y = np.log(train_data['SalePrice'])

In [0]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [0]:
from sklearn.linear_model import LinearRegression
residuals = y_train - LinearRegression().fit(X_train, y_train).predict(X_train)
outliers = residuals[np.abs(zscore(residuals)) > 3].index
outliers

In [0]:
X_train = X_train.drop(outliers)
y_train = y_train.drop(outliers)

In [0]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
clf.fit(X_train, y_train)
from sklearn.metrics import mean_squared_error
from math import sqrt
y_pred = clf.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print(rms)

In [0]:
import xgboost
# from sklearn import ensemble
# params = {'n_estimators': 500, 'max_depth': 3, 'min_samples_split': 2,
#           'learning_rate': 0.01, 'loss': 'ls'}
clf = xgboost.XGBRegressor()
clf.fit(X_train, y_train)
from sklearn.metrics import mean_squared_error
from math import sqrt

y_pred = clf.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print(rms)

In [0]:
null_table = pd.DataFrame(dicto)
null_table.sort_values(by='null_values', ascending=False)

In [0]:
df.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu','MasVnrType'], axis=1, inplace=True)

In [0]:
null_table.drop(null_table.nlargest(6, columns='null_values').index,axis=0, inplace=True)

In [0]:
null_table

In [0]:
cols = ["GarageCond", "GarageQual", "GarageFinish", "GarageType", "BsmtCond", "BsmtExposure", "BsmtQual", "BsmtFinType2", "BsmtFinType1", "Electrical"]
df[cols] = df.groupby("Neighborhood")[cols].transform(lambda x: x.fillna(x.mode()[0]))

# cols = ["GarageYrBlt", "MasVnrArea", "BsmtHalfBath", "BsmtFullBath", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "GarageCars"]
# X[cols] = X[cols].fillna(0)
# cols = ["MSZoning", "Utilities", "Exterior1st", "Exterior2nd", "SaleType", "Electrical", "KitchenQual", "Functional"]
# X[cols] = X.groupby("Neighborhood")[cols].transform(lambda x: x.fillna(x.mode()[0]))
# cols = ["GarageArea", "LotFrontage"]
# X[cols] = X.groupby("Neighborhood")[cols].transform(lambda x: x.fillna(x.median()))
# Engineer features

###deleting the five columns
PoolQC          

MiscFeature     

Alley           

Fence           

FireplaceQu      

MasVnrType


In [0]:
#now for the continous variables
w=df.isnull().sum().sort_values(ascending=False)
w[w>0]
cols = ["GarageYrBlt", "LotFrontage","MasVnrArea"]
df[cols] = df.groupby("Neighborhood")[cols].transform(lambda x: x.fillna(x.median()))

In [0]:
w=df.isnull().sum().sort_values(ascending=False)
w[w>0]


In [0]:
# df = df.merge(pd.get_dummies(data=df, columns=['BsmtQual', 'GarageType']))


In [0]:
# import seaborn as sns
# plt.figure(figsize=(50,50))
# corr = df.corr()
# sns.heatmap(corr, annot=True)

In [0]:
X = df.loc[:, df.columns!='SalePrice']

y = np.log(df['SalePrice'])

In [0]:
# from sklearn.preprocessing import StandardScaler
# p = StandardScaler()
# X = p.fit_transform(X)

In [0]:
#train split 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [0]:
from sklearn.linear_model import LinearRegression
residuals = y_train - LinearRegression().fit(X_train, y_train).predict(X_train)
outliers = residuals[np.abs(zscore(residuals)) > 3].index

In [0]:
outliers

In [0]:
X_train.drop(outliers, inplace=True)
y_train.drop(outliers, inplace=True)

In [0]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)
rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))
scorer = make_scorer(rmse, greater_is_better=False)
def random_search(model, grid, n_iter=100):
    n_jobs = max(cpu_count() -1, 1)
    search = RandomizedSearchCV(model, grid, n_iter, scorer, n_jobs=n_jobs, cv=kf, random_state=0, verbose=True)
    return search.fit(X_train, y_train)
ridge_search = random_search(Ridge(), {"alpha": np.logspace(-1, 2, 500)})
lasso_search = random_search(Lasso(), {"alpha": np.logspace(-5, -1, 500)})
svr_search = random_search(SVR(), {"C": np.arange(1, 100), "gamma": np.linspace(0.00001, 0.001, 50), "epsilon": np.linspace(0.01, 0.1, 50)})
lgbm_search = random_search(LGBMRegressor(n_estimators=2000, max_depth=3), {"colsample_bytree": np.linspace(0.2, 0.7, 6), "learning_rate": np.logspace(-3, -1, 100)})
gbm_search = random_search(GradientBoostingRegressor(n_estimators=2000, max_depth=3), {"max_features": np.linspace(0.2, 0.7, 6), "learning_rate": np.logspace(-3, -1, 100)})
models = [search.best_estimator_ for search in [ridge_search, lasso_search, svr_search, lgbm_search, gbm_search]]
stack_search = random_search(StackingCVRegressor(models, Ridge(), cv=kf), {"meta_regressor__alpha": np.logspace(-3, -2, 500)}, n_iter=20)
models.append(stack_search.best_estimator_)
preds = [model.predict(X_test) for model in models]


In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt

for i in preds:
  rms = sqrt(mean_squared_error(y_test, i))

  print(rms)

In [0]:
y_test.values-preds[4]

In [0]:
import xgboost
# from sklearn import ensemble
# params = {'n_estimators': 500, 'max_depth': 3, 'min_samples_split': 2,
#           'learning_rate': 0.01, 'loss': 'ls'}
clf = xgboost.XGBRegressor()
clf.fit(X_train, y_train)
from sklearn.metrics import mean_squared_error
from math import sqrt
y_pred = clf.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print(rms)

In [0]:
corr = df.corr()
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
selected_columns = df.columns[columns]
data = df[selected_columns]
selected_columns = selected_columns[0:-1].values


In [0]:
X.shape, y.shape,len(selected_columns), df.shape

In [0]:

import statsmodels.api as sm
def backwardElimination(x, Y, sl, columns):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(Y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
                    columns = np.delete(columns, j)
                    
    regressor_OLS.summary()
    return x, columns
SL = 0.5
data_modeled, selected_columns = backwardElimination(data.iloc[:,1:].values, data.iloc[:,0].values, SL, selected_columns)

In [0]:
data = pd.DataFrame(data = data_modeled, columns = selected_columns)


In [0]:
data.head()

In [0]:
from sklearn.preprocessing import StandardScaler
p = StandardScaler()
X = p.fit_transform(data)

In [0]:
#train split 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data,y, test_size=0.2, random_state=0)

In [0]:
#create model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt
y_pred = model.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print(rms)

In [0]:
from sklearn.linear_model import Lasso
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 3, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
clf.fit(X_train, y_train)
from sklearn.metrics import mean_squared_error
from math import sqrt
y_pred = clf.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print(rms)

In [0]:
import xgboost
# from sklearn import ensemble
# params = {'n_estimators': 500, 'max_depth': 3, 'min_samples_split': 2,
#           'learning_rate': 0.01, 'loss': 'ls'}
clf = xgboost.XGBRegressor()
clf.fit(X_train, y_train)
from sklearn.metrics import mean_squared_error
from math import sqrt
y_pred = clf.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print(rms)